<a href="https://colab.research.google.com/github/hoo0681/raspi4withTF/blob/master/model_DEV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 기본세팅

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
 %tensorflow_version 2.x

import tensorflow as tf

import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import pathlib
print("Module Loaded.")
print("TensorFlow Version :{}".format(tf.__version__))
print("NumPy Version :{}".format(np.__version__))
print("Matplotlib Version :{}".format(plt.matplotlib.__version__))

# set auto tune
AUTOTUNE = tf.data.experimental.AUTOTUNE

TensorFlow 2.x selected.
Module Loaded.
TensorFlow Version :2.1.0-rc1
NumPy Version :1.17.4
Matplotlib Version :3.1.2


In [0]:
# tf.keras.preprocessing.image.ImageDataGenerator()
## 가 생성한 결과의 일부를 확인
def show_batch(data_gen, class_l):
  # get image and label from data generator
    img_batch, l_batch = next(data_gen) 
    plt.figure(figsize=(10,10))
    for n in range(25):
        ax = plt.subplot(5,5,n+1)
        plt.imshow(img_batch[n])
        plt.title(class_l[l_batch[n]==1][0].title())
        plt.axis('off')
    return img_batch, l_batch

In [0]:
# 지정 폴더 아래에 있는 모든 *.jpg 파일의 수
#  및 폴더명 목록을 리턴  
def check_dir(d_path):
    img_count = len(list(d_path.glob('*/*.jpg')))
    c_name = np.array([item.name for item in d_path.glob('*') if item.name != "LICENSE.txt"])
    return img_count, c_name

In [0]:
# 지정 path 아래에 있는 폴더에서 이미지 두장씩을 확인
def check_image(d_path, class_list):
    for i in range(len(class_list)):
        class_temp = list(d_path.glob(str(class_list[i])+'/*'))
        for image_path in class_temp[:2]:
            display.display(Image.open(str(image_path)))

In [0]:
# image data generator로 부터 image를 받아서 출력 
def k_image_process(dg, imgs):
  # batch_size 만큼씩 처리하도록 준비
    it = dg.flow(imgs, batch_size=1)
    plt.figure(figsize=(12,9))
    # generate samples and plot
    for i in range(9):
        # define subplot
        plt.subplot(330 + 1 + i)
        #batch_size씩 가져옴
        batch = it.next() 
        # plt을 위해 type 변경 
        image = batch[0].astype('uint8')
        # plot raw pixel data
        plt.imshow(image)
  # show the figure
    plt.show()

# 실패한 data loading

In [0]:
data_dir_train = pathlib.Path('/content/drive/My Drive/tf2/z_data/asirra/train')
data_dir_test = pathlib.Path('/content/drive/My Drive/tf2/z_data/asirra/test')

image_count_gd, CLASS_NAMES_gd = check_dir(data_dir_train)
# image generator for unziped directory 
# The 1./255 is to convert from uint8 to float32 in range [0,1].
image_generator_train = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)#,
#                                                                        shear_range=0.2,
 #                                                                       zoom_range=0.2,
   #                                                                     horizontal_flip=True)
image_generator_test=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# generate train dataset
train_data_gen_gd = image_generator_train.flow_from_directory(
                    directory=str(data_dir_train),
                    batch_size=250,
                    shuffle=True,
                    target_size=(128, 128),
                    class_mode='categorical',
                    classes = list(CLASS_NAMES_gd))
test_data_gen_gd=image_generator_test.flow_from_directory(
                    directory=str(data_dir_test),
                    batch_size=30,
                    target_size=(128, 128),
                    class_mode='categorical',
                    classes = list(CLASS_NAMES_gd))
# check dataset
#show_batch(test_data_gen_gd, CLASS_NAMES_gd)

Found 12510 images belonging to 2 classes.
Found 12520 images belonging to 2 classes.


In [0]:
%%time
train_data,train_y=train_data_gen_gd.next()

CPU times: user 773 ms, sys: 71.6 ms, total: 845 ms
Wall time: 43.3 s


# data loading

In [0]:
cwd = os.getcwd()
cwd

'/content'

In [0]:
!unzip /content/drive/My\ Drive/tf2/z_data/asirra_fixed.zip

In [0]:
!ls -al asirra/test

In [0]:
import pathlib
train_data_dir_cwd=pathlib.Path('/content/asirra/train')
test_data_dir_cwd=pathlib.Path('/content/asirra/test')

In [0]:
image_count_cwd, CLASS_NAMES_cwd = check_dir(train_data_dir_cwd)
test_image_count_cwd, CLASS_NAMES_cwd = check_dir(test_data_dir_cwd)
print('image_count: {}\nclasses: {}'.format(image_count_cwd, CLASS_NAMES_cwd))
print('image_count: {}\nclasses: {}'.format(test_image_count_cwd, CLASS_NAMES_cwd))

In [0]:
check_image(train_data_dir_cwd, CLASS_NAMES_cwd)

In [0]:
# image generator 생성
# 1./255 : convert in range [0,1]
image_generator_cwd = (
    tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255))

In [0]:
train_data_gen_gd = image_generator_cwd.flow_from_directory(
                        directory=str(train_data_dir_cwd),
                        batch_size=512,
                        shuffle=True,
                        target_size=(112, 112),
                        classes = list(CLASS_NAMES_cwd))
test_data_gen_gd  = image_generator_cwd.flow_from_directory(
                        directory=str(test_data_dir_cwd),
                        batch_size=512,
                        shuffle=True,
                        target_size=(112, 112),
                        classes = list(CLASS_NAMES_cwd))

In [0]:
show_batch(train_data_gen_gd, CLASS_NAMES_cwd)

# 모델정의

In [0]:
row,col,pixel=test_data_gen_gd.image_shape
num_classes = test_data_gen_gd.num_classes
input_shape=test_data_gen_gd.image_shape


In [0]:
import tensorflow.keras.layers as layers
from tensorflow.keras import models
def model_function():
    input_= layers.Input(shape=(input_shape))                   # (28,28,1)
    x=layers.Conv2D(64,3,padding='same',activation='relu',name='conv32')(input_)
    x=layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(x)
    x=layers.Conv2D(32,3,padding='same',activation='relu',name='conv32_2')(x)
    x=layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(x)
    x=layers.Conv2D(16,3,padding='same',activation='relu',name='conv32_3')(x)
    x=layers.Flatten()(x)
    prediction = layers.Dense(num_classes,activation='softmax')(x) # (?,10)<-(?,30)
    # return model
    return models.Model(input_, prediction)

In [40]:
model = model_function()


model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.summary()


Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 112, 112, 3)]     0         
_________________________________________________________________
conv32 (Conv2D)              (None, 112, 112, 64)      1792      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 56, 56, 64)        0         
_________________________________________________________________
conv32_2 (Conv2D)            (None, 56, 56, 32)        18464     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 28, 28, 32)        0         
_________________________________________________________________
conv32_3 (Conv2D)            (None, 28, 28, 16)        4624      
_________________________________________________________________
flatten_2 (Flatten)          (None, 12544)             0   

In [0]:
%%time
train_data,train_y=train_data_gen_gd.next()

CPU times: user 948 ms, sys: 94.4 ms, total: 1.04 s
Wall time: 1.04 s


# 학습

In [0]:
%%time
history=model.fit(train_data_gen_gd,epochs=6,steps_per_epoch=10,validation_data=test_data_gen_gd)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 10 steps, validate for 25 steps
Epoch 1/6
10/10 [==============================] - 131s 13s/step - loss: 0.6658 - accuracy: 0.6055 - val_loss: 0.6596 - val_accuracy: 0.5949
Epoch 2/6
10/10 [==============================] - 131s 13s/step - loss: 0.6229 - accuracy: 0.6420 - val_loss: 0.6113 - val_accuracy: 0.6578
Epoch 3/6
10/10 [==============================] - 130s 13s/step - loss: 0.6976 - accuracy: 0.6031 - val_loss: 0.6428 - val_accuracy: 0.6397
Epoch 4/6
10/10 [==============================] - 132s 13s/step - loss: 0.6013 - accuracy: 0.6854 - val_loss: 0.5995 - val_accuracy: 0.6811
Epoch 5/6
10/10 [==============================] - 131s 13s/step - loss: 0.5999 - accuracy: 0.6578 - val_loss: 0.5860 - val_accuracy: 0.6958
Epoch 6/6
10/10 [==============================] - 131s 13s/step - loss: 0.6118 - accuracy: 0.6602 - val_loss: 0.6010 - val_accuracy: 0.6719
CPU times: user 45min 23s, sys: 38.1 s, total: 46min 2s
Wall t

# 평가

In [0]:
loss = history.history['loss']
epochs = range(1, len(loss)+1)

plt.figure(figsize=(6, 10))
plt.subplot(2, 1, 1)
plt.title('Accuray')
plt.plot(epochs, history.history['accuracy'], 'b', label='train_accuracy')
plt.plot(epochs, history.history['val_accuracy'], 'g', label='val_accuracy')

plt.grid(True)
plt.ylabel('Accuracy')
plt.ylim([0.5, 0.8])
plt.legend(loc='best')

plt.subplot(2, 1, 2)
plt.title('Loss')
plt.plot(epochs, history.history['loss'], 'b', label='train_loss')
plt.plot(epochs, history.history['val_loss'], 'g', label='val_loss')
plt.grid(True)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim([0.0, 0.9])
plt.legend(loc='best')
plt.show()

In [0]:
img=tf.keras.preprocessing.image.load_img('/content/asirra/train/cat/cat.4338.jpg')

In [39]:
tf.keras.preprocessing.image.img_to_array(img)

(<module 'numpy' from '/usr/local/lib/python3.6/dist-packages/numpy/__init__.py'>,)

# 저장

In [0]:
model.save('my_model.hdf5')
# delete
del model

In [0]:
from tensorflow.keras.models import load_model
model=load_model('my_model.hdf5')


{'accuracy': [0.5980815],
 'loss': [0.6746060273630156],
 'val_accuracy': [0.67],
 'val_loss': [0.5963890075683593]}